# 关系抽取

## 一、演练介绍
### 1. 演练内容
在上一演练中，将实体从文本中抽取出来，在实体抽取出来之后得到的是离散的节点，为了构成网状的知识，还需要从文本中提取实体之间的关系。因此，本演练中将演练如何从文本中抽取实体之间的关系。
### 2. 演练技能点
在本演练中，将接触
- 关系抽取任务的定义与理论解决方法
- 关系抽取任务中数据集处理方法
- 关系抽取模型——PCNN 的理论知识
- 用 Keras 构建 PCNN
- PCNN 的训练与预测方法
### 3. 演练要求
本演练要求具备以下基本能力：
- Keras 的基本使用方法

## 二、演练原理

### 1. 关系抽取任务

在本演练中，关心的是有监督的关系抽取任务，即已知所有文本中包含的关系种类。此时关系抽取的任务形式就是一个文本分类的问题——任务的输入是一句话以及这句话中包含的两个实体，输出是关系类别。

如文本“杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。”中，一共有四个人名实体，要获得“杨康”与“杨铁心”的关系，那么就要把“杨康”，“杨铁心”，“杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。” 这三个数据都输入到算法中。

### 2. 数据预处理
首先对数据进行位置编码，按句子中各个词离实体的距离进行编码。
如“杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。”中，实体为“杨康”和“杨铁心”。然后记录句子中每个字与实体首字之间的距离。

如

|杨|康|，|杨|铁|心|与|包|惜|弱|之|子|，|金|国|六|王|爷|完|颜|洪|烈|的|养|子|。|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|

`pos_1=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]`，0 就是杨康的起始位置

|杨|康|，|杨|铁|心|与|包|惜|弱|之|子|，|金|国|六|王|爷|完|颜|洪|烈|的|养|子|。|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|-3|-2|-1|0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|

`pos_2=[-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]`，0 就是杨铁心的起始位置



## 三、演练步骤 
先安装需要的包：

In [4]:
# 如果已经安装过，可以不用再次安装，略过此步骤
#!pip install tensorflow==1.13.1
#!pip install keras==2.1.6

### 1. 数据处理
原始文本和标签定义为

In [5]:
# 对于 lists 中每一个子列表，第一个元素为实体1，第二个元素为实体2，第三个元素为实体1对实体2的关系，第四个元素为文本。
lists = [['杨康','杨铁心','子女','杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
         ['杨康','杨铁心','子女','丘处机与杨铁心、郭啸天结识后，以勿忘“靖康之耻”替杨铁心的儿子杨康取名。'],
         ['杨铁心','包惜弱','配偶','金国六王爷完颜洪烈因为贪图杨铁心的妻子包惜弱的美色，杀害了郭靖的父亲郭啸天。'],
         ['杨铁心','包惜弱','配偶','杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
         ['张翠山','殷素素','配偶','张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。'],
         ['小龙女','杨过','师傅','小龙女是杨过的师父，与杨过互生情愫，但因师生恋不容于世。'],
         ['黄药师','黄蓉','父','黄药师，黄蓉之父，对其妻冯氏（小字阿衡）一往情深。'],
         ['郭啸天','郭靖','父','郭靖之父郭啸天和其义弟杨铁心因被段天德陷害，死于临安牛家村。']]

relation2idx = {'子女':0,'配偶':1,'师傅':2,'父':3}

lists, relation2idx

([['杨康', '杨铁心', '子女', '杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
  ['杨康', '杨铁心', '子女', '丘处机与杨铁心、郭啸天结识后，以勿忘“靖康之耻”替杨铁心的儿子杨康取名。'],
  ['杨铁心', '包惜弱', '配偶', '金国六王爷完颜洪烈因为贪图杨铁心的妻子包惜弱的美色，杀害了郭靖的父亲郭啸天。'],
  ['杨铁心', '包惜弱', '配偶', '杨康，杨铁心与包惜弱之子，金国六王爷完颜洪烈的养子。'],
  ['张翠山', '殷素素', '配偶', '张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。'],
  ['小龙女', '杨过', '师傅', '小龙女是杨过的师父，与杨过互生情愫，但因师生恋不容于世。'],
  ['黄药师', '黄蓉', '父', '黄药师，黄蓉之父，对其妻冯氏（小字阿衡）一往情深。'],
  ['郭啸天', '郭靖', '父', '郭靖之父郭啸天和其义弟杨铁心因被段天德陷害，死于临安牛家村。']],
 {'子女': 0, '配偶': 1, '师傅': 2, '父': 3})

首先，先将 `lists` 中的实体、关系与文本都单独拆分开来，并对文本进行位置编码。

In [6]:
datas, labels, pos_list1, pos_list2 = [], [], [], []
translation = 32
for entity1, entity2, relation, text in lists:
    # 找到第一个实体出现的下标
    idx1 = text.index(entity1)
    # 找到第二个实体出现的下标
    idx2 = text.index(entity2)
    sentence, pos1, pos2 = [], [], []
    for i, w in enumerate(text):
        sentence.append(w)
        # 计算句子中每个字与实体1首字的距离
        pos1.append(i-idx1+translation)
        # 计算句子中每个字与实体2首字的距离
        pos2.append(i-idx2+translation)
    datas.append(sentence)
    labels.append(relation2idx[relation])
    pos_list1.append(pos1)
    pos_list2.append(pos2)

datas, labels, pos_list1, pos_list2

([['杨',
   '康',
   '，',
   '杨',
   '铁',
   '心',
   '与',
   '包',
   '惜',
   '弱',
   '之',
   '子',
   '，',
   '金',
   '国',
   '六',
   '王',
   '爷',
   '完',
   '颜',
   '洪',
   '烈',
   '的',
   '养',
   '子',
   '。'],
  ['丘',
   '处',
   '机',
   '与',
   '杨',
   '铁',
   '心',
   '、',
   '郭',
   '啸',
   '天',
   '结',
   '识',
   '后',
   '，',
   '以',
   '勿',
   '忘',
   '“',
   '靖',
   '康',
   '之',
   '耻',
   '”',
   '替',
   '杨',
   '铁',
   '心',
   '的',
   '儿',
   '子',
   '杨',
   '康',
   '取',
   '名',
   '。'],
  ['金',
   '国',
   '六',
   '王',
   '爷',
   '完',
   '颜',
   '洪',
   '烈',
   '因',
   '为',
   '贪',
   '图',
   '杨',
   '铁',
   '心',
   '的',
   '妻',
   '子',
   '包',
   '惜',
   '弱',
   '的',
   '美',
   '色',
   '，',
   '杀',
   '害',
   '了',
   '郭',
   '靖',
   '的',
   '父',
   '亲',
   '郭',
   '啸',
   '天',
   '。'],
  ['杨',
   '康',
   '，',
   '杨',
   '铁',
   '心',
   '与',
   '包',
   '惜',
   '弱',
   '之',
   '子',
   '，',
   '金',
   '国',
   '六',
   '王',
   '爷',
   '完',
   '颜',
   '洪',
   '烈',
   '的',
   '养',
   '子

In [7]:
from collections import Counter
# 统计每个字出现的次数, sum(datas,[]) 的功能是将列表铺平
word_counts = Counter(sum(datas, []))
# 建立字典表，只记录出现次数不小于 2 的字
vocab = [w for w, f in iter(word_counts.items()) if f >= 2]
word_counts, vocab

(Counter({'杨': 11,
          '康': 4,
          '，': 11,
          '铁': 6,
          '心': 6,
          '与': 5,
          '包': 3,
          '惜': 3,
          '弱': 3,
          '之': 7,
          '子': 7,
          '金': 3,
          '国': 3,
          '六': 3,
          '王': 3,
          '爷': 3,
          '完': 3,
          '颜': 3,
          '洪': 3,
          '烈': 3,
          '的': 7,
          '养': 2,
          '。': 8,
          '丘': 1,
          '处': 1,
          '机': 1,
          '、': 1,
          '郭': 5,
          '啸': 3,
          '天': 5,
          '结': 1,
          '识': 1,
          '后': 1,
          '以': 1,
          '勿': 1,
          '忘': 1,
          '“': 1,
          '靖': 3,
          '耻': 1,
          '”': 1,
          '替': 1,
          '儿': 1,
          '取': 1,
          '名': 1,
          '因': 3,
          '为': 1,
          '贪': 1,
          '图': 1,
          '妻': 2,
          '美': 1,
          '色': 1,
          '杀': 1,
          '害': 2,
          '了': 1,
          '父': 4,
        

In [8]:
# 构建词袋模型，和上一演练相同，将字典从 2 开始编号，把 0 和 1 空出来，0 作为填充元素，1 作为不在字典中的字的编号
word2idx = dict((w,i+2) for i,w in enumerate(vocab))
word2idx

{'杨': 2,
 '康': 3,
 '，': 4,
 '铁': 5,
 '心': 6,
 '与': 7,
 '包': 8,
 '惜': 9,
 '弱': 10,
 '之': 11,
 '子': 12,
 '金': 13,
 '国': 14,
 '六': 15,
 '王': 16,
 '爷': 17,
 '完': 18,
 '颜': 19,
 '洪': 20,
 '烈': 21,
 '的': 22,
 '养': 23,
 '。': 24,
 '郭': 25,
 '啸': 26,
 '天': 27,
 '靖': 28,
 '因': 29,
 '妻': 30,
 '害': 31,
 '父': 32,
 '张': 33,
 '一': 34,
 '素': 35,
 '小': 36,
 '过': 37,
 '师': 38,
 '生': 39,
 '情': 40,
 '于': 41,
 '黄': 42,
 '其': 43}

In [9]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

# 构建输入，即对于样本中每一个字，从词袋模型中找到这个字对应的 idx，出现频率过低的字，并没有出现在词袋模型中，此时将这些字的 idx 取为 1
train_x = [[word2idx.get(w, 1) for w in s] for s in datas]

max_len = 64

# 在输入的左边填充 0
train_x = pad_sequences(train_x, max_len, value=0)
## 填充位置编码
train_pos1 = pad_sequences(pos_list1, max_len, value=0)
train_pos2 = pad_sequences(pos_list2, max_len, value=0)
# one_hot 编码 label
train_y = to_categorical(labels, num_classes=len(relation2idx))

train_x.shape, train_y.shape, train_pos1.shape, train_pos2.shape

Using TensorFlow backend.
f:\Venvs\kg01\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
f:\Venvs\kg01\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
f:\Venvs\kg01\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
f:\Venvs\kg01\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1

((8, 64), (8, 4), (8, 64), (8, 64))

### 2. 构建网络模型
因为网络有多个输入：文本与位置编码，属于复杂模型，因此这里使用 Keras 的函数式 API 来定义网络结构

In [10]:
from keras.layers import Input, Embedding, concatenate, Conv1D, GlobalMaxPool1D, Dense, LSTM
from keras.models import Model

# 定义输入层
words = Input(shape=(max_len,),dtype='int32')
position1 = Input(shape=(max_len,),dtype='int32')
position2 = Input(shape=(max_len,),dtype='int32')
#  Embedding 层将输入进行编码
pos_emb1 = Embedding(output_dim=16, input_dim=256)(position1)
pos_emb2 = Embedding(output_dim=16, input_dim=256)(position2)
word_emb = Embedding(output_dim=16, input_dim=256)(words)
# 分别拼接 文本编码与位置1 和文本编码与位置2
concat1 = concatenate([word_emb, pos_emb1])
concat2 = concatenate([word_emb, pos_emb2])
# 卷积池化层
conv1 = Conv1D(filters=128, kernel_size=3)(concat1)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=3)(concat2)
pool2 = GlobalMaxPool1D()(conv2)
# 拼接，最后接全连接层，激活函数为 softmax
concat = concatenate([pool1, pool2])
out = Dense(units=len(relation2idx),activation='softmax')(concat)

model = Model(inputs=[words, position1, position2],outputs=out)
# 编译模型
model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 64, 16)       4096        input_1[0][0]                    
_____________________________________

In [11]:
# 训练 50 次
model.fit([train_x, train_pos1, train_pos2], train_y, batch_size=8, epochs=50)
model.save('model.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
8/8 [==============================] - 0s 34ms/step - loss: 1.3853 - acc: 0.2500
Epoch 2/50
8/8 [==============================] - 0s 487us/step - loss: 1.3583 - acc: 0.7500
Epoch 3/50
8/8 [==============================] - 0s 375us/step - loss: 1.3339 - acc: 0.8750
Epoch 4/50
8/8 [==============================] - 0s 312us/step - loss: 1.3115 - acc: 0.7500
Epoch 5/50
8/8 [==============================] - 0s 526us/step - loss: 1.2906 - acc: 0.5000
Epoch 6/50
8/8 [==============================] - 0s 499us/step - loss: 1.2708 - acc: 0.5000
Epoch 7/50
8/8 [==============================] - 0s 376us/step - loss: 1.2521 - acc: 0.5000
Epoch 8/50
8/8 [==============================] - 0s 499us/step - loss: 1.2340 - acc: 0.5000
Epoch 9/50
8/8 [==============================] - 0s 376us/step - loss: 1.2163 - acc: 0.3750
Epoch 10/50
8/8 [=====================

### 3. 模型预测 
在这里使用训练集中的一个实例进行预测

In [12]:
test_instance = ['张翠山','殷素素','张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。']
test_ne1, test_ne2, test_text = test_instance
test_ne1, test_ne2, test_text

('张翠山', '殷素素', '张无忌,武当七侠之一张翠山与天鹰教紫微堂主殷素素之子。')

In [13]:
# 将预测数据转换为向量
pred_x = [word2idx.get(w, 1) for w in test_text]
idx1 = test_text.index(test_ne1)
idx2 = test_text.index(test_ne2)
pos1 = [i-idx1+translation for i in range(len(test_text))]
pos2 = [i-idx2+translation for i in range(len(test_text))]
pred_x = pad_sequences([pred_x], max_len, value=0)
test_pos1 = pad_sequences([pos1], max_len, value=0)
test_pos2 = pad_sequences([pos2], max_len, value=0)
pred_x, test_pos1, test_pos2

(array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0, 33,  1,  1,  1,  1,  1,  1,  1, 11, 34, 33,
          1,  1,  7, 27,  1,  1,  1,  1,  1,  1,  1, 35, 35, 11, 12, 24]]),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
         33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]]),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
         22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]]))

In [14]:
# 翻转 relation2idx 字典
idx2relation = dict(zip(relation2idx.values(),relation2idx.keys()))
# 使用模型进行预测
pred = model.predict([pred_x, test_pos1, test_pos2])
# 模型预测最大值的位置作为预测值
output_idx = np.argmax(pred)
# 找到 idx2relation 中实际的标签
output_label = idx2relation[output_idx]
pred, output_idx, output_label

(array([[0.07006253, 0.7985946 , 0.04585258, 0.08549027]], dtype=float32),
 1,
 '配偶')

## 4. 总结 
在本演练中，演示了关系抽取任务的定义，并使用了一个小型的神经网络来实现关系抽取任务中的数据处理、训练与预测。

由于小型的神经网络的参数量较少，拟合能力有限，从而随着训练数据量的增加就会出现欠拟合的现象。因此，用少量数据集在小型网络上训练完成后，再逐渐增大数据量，同时将小型网络复杂化，如使用现有的 [PCNN](http://www.emnlp2015.org/proceedings/EMNLP/pdf/EMNLP203.pdf)，[Attention-BiLSTM](https://www.aclweb.org/anthology/P16-2034) 等用于关系抽取的经典神经网络结构，将任务的精度提升到想要的结果。